<a href="https://colab.research.google.com/github/DaniloLustosa-eng/Projeto-BigData/blob/main/big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
#Pacotes
!pip install -q pyspark
!sudo apt update
!sudo apt install openjdk-11-jdk -y
!pip install transformers
!pip install torch
!pip install huggingface_hub[hf_xet]
!pip install tiktoken protobuf sentencepiece


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://cli.github.com/packages stable InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
43 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as re

In [48]:
#Imports
import kagglehub
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import lower, col
from transformers import pipeline

In [49]:

path_1 = kagglehub.dataset_download("ayushijain12345/tweeter")
path_2 = kagglehub.dataset_download("augustop/portuguese-tweets-for-sentiment-analysis")
path_3= kagglehub.dataset_download("derrickmwiti/24-thousand-tweets-later")


Using Colab cache for faster access to the 'tweeter' dataset.
Using Colab cache for faster access to the 'portuguese-tweets-for-sentiment-analysis' dataset.
Using Colab cache for faster access to the '24-thousand-tweets-later' dataset.


In [50]:

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] += ":/usr/lib/jvm/java-11-openjdk-amd64/bin"

In [51]:

spark = SparkSession.builder.appName("Spark-1").getOrCreate()

In [52]:
df_1= spark.read.csv(path_1, header=True, inferSchema=True)
df_2= spark.read.csv(path_2, header=True, inferSchema=True)
df_3= spark.read.csv(path_3, header=True, inferSchema=True)

In [53]:

df_normalizado_1 = df_1.withColumn("tweet", lower(col("tweet")))
df_normalizado_2 = df_2.withColumn("tweet_text", lower(col("tweet_text")))
df_normalizado_3 = df_3.withColumn("tweet", lower(col("tweet ")))

In [54]:

df_filtrado_1 = df_normalizado_1.filter(col("tweet").rlike(r"\b(java|python|javascript|ruby|php)\b"))
df_filtrado_2 = df_normalizado_2.filter(col("tweet_text").rlike(r"\b(java|python|javascript|ruby|php)\b"))
df_filtrado_3 = df_normalizado_3.filter(col("tweet").rlike(r"\b(java|python|javascript|ruby|php)\b"))

In [55]:
colunas = df_filtrado_3.columns
# Remove a sexta coluna
colunas_sem_sexta = [c for i, c in enumerate(colunas) if i != 5]
# Cria um novo DataFrame sem a sexta coluna
df_definitivos_3 = df_filtrado_3.select(*colunas_sem_sexta)


In [56]:
#data-frames normalizados e com uma unica coluna
df_final_1 = df_filtrado_1.select("tweet")
df_final_2_errado = df_filtrado_2.select("tweet_text")
df_final_2 = df_final_2_errado.withColumnRenamed("tweet_text", "tweet")
df_final_3 = df_filtrado_3.select("tweet")

In [57]:
df_final_unido = df_final_1.union(df_final_2).union(df_final_3)

In [58]:

tweets = df_final_unido.select("tweet").rdd.map(lambda x: x[0]).collect()
sentimentos = []
classifier = pipeline("sentiment-analysis",model="cardiffnlp/twitter-xlm-roberta-base-sentiment")
for t in tweets:
  sentimentos.append(classifier(t)[0]['label'])

Device set to use cpu


In [59]:
data_frame_final_com_sentimentos = spark.createDataFrame(zip(tweets,sentimentos),["tweet","sentimento"])

In [60]:
data_frame_final_com_sentimentos.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("df_final_unido.csv")
